In [10]:
# Experiment Settings, logger, plotter
from utils.exp_logger import Logger
from utils.exp_metrics_plotter import MetricsPlotter
from utils.utils import set_settings
from utils.exp_config import get_config

config = get_config("OurModelConfig")
set_settings(config)

<module 'module.name' from '/home/rtx4090/code/python/current/LightNAS/configs/OurModelConfig.py'> OurModelConfig
✅ All __pycache__ folders removed


OurModelConfig(classification=False, ablation=0, try_exp=1, ts_var=1, input_size=192, bs=256, lr=0.001, decay=0.0001, loss_func='MSELoss', optim='Adam', epochs=2000, patience=200, verbose=2000, device='cuda', use_amp=False, monitor_metric='KendallTau', monitor_reverse=True, path='./data', task='bench201', dataset='101_acc', predict_target='accuracy', eval_set=True, shuffle=False, scaler_method='minmax', spliter_ratio='5:4:91', sample_method='random', logger='zyx', model='ours', d_model=192, num_layers=4, retrain=True, seed=0, rounds=5, runid=0, debug=False, record=True, hyper_search=False, continue_train=False, op_encoder='embedding', lp_d_model=8, lap_node_id_sign_flip=True, num_heads=4, att_method='self', att_bias=True, transfer=False, rank_loss=True, ac_loss=True)

In [18]:
data_root = './data/nnlqp/unseen_structure/'
all_latency_file = data_root + "/gt.txt"
test_model_type = ['resnet18' 'vgg16' 'efficientb0' 'mobilenetv2' 'mobilenetv3' 'mnasnet' 'alexnet' 'squeezenet' 'googlenet' 'nasbench201']

In [19]:
model_types = set()
for line in open(all_latency_file).readlines():
    model_types.add(line.split()[4])
assert test_model_type in model_types
test_model_types = set([test_model_type])
if train_model_types:
    train_model_types = set(train_model_types.split(','))
    train_model_types = train_model_types & model_types
else:
    train_model_types = model_types - test_model_types
assert len(train_model_types) > 0
allow_platforms=multi_plt.keys() if multi_plt else None

self.logger.info("Train model types: {}".format(train_model_types))
self.logger.info("Test model types: {}".format(test_model_types))
self.logger.info("Platforms: {}".format(allow_platforms))

train_set = GraphLatencyDataset(
    data_root,
    onnx_dir,
    all_latency_file,
    override_data=override_data,
    model_types=train_model_types,
    platforms=allow_platforms,
)
test_set = GraphLatencyDataset(
    data_root,
    onnx_dir,
    all_latency_file,
    override_data=override_data,
    model_types=test_model_types,
    platforms=allow_platforms,
)

self.logger.info("Train data = {}, Test data = {}".format(len(train_set), len(test_set)))
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False)

TypeError: unhashable type: 'list'

In [ ]:

"""
    俩个场景
    1、Out of domain latency prediction on NNLQ
    2、In domain latency prediction on NNLQ
"""
trainset, testset, trtypes, tetypes = init_dataset_NNLQP(
    config.data_path,
    config.test_model_type,
    config.override_data,
    config.embed_type,
    config.finetuning,
)
print("Train model types: {}".format(trtypes))
print("Test model types: {}".format(tetypes))

train_sampler = FixedLengthBatchSampler(
    trainset, config.dataset, config.batch_size, include_partial=True
)
test_sampler = FixedLengthBatchSampler(
    testset, config.dataset, config.batch_size, include_partial=True
)

train_loader = DataLoader(
    trainset,
    shuffle=(train_sampler is None),
    num_workers=config.n_workers,
    pin_memory=True,
    persistent_workers=True,
    batch_sampler=train_sampler,
)
test_loader = DataLoader(
    testset,
    shuffle=(test_sampler is None),
    num_workers=config.n_workers,
    pin_memory=True,
    persistent_workers=True,
    batch_sampler=test_sampler,
)

AttributeError: 'OurModelConfig' object has no attribute 'data_path'

In [ ]:

"""
    mobile 角度？
    cpu -> gpu tpu dsp
    gpu -> cpu tpu dsp
    tpu -> cpu gpu dsp
"""


def get_bench201_latency(config):
    with open("./data/nasbench201/pkl/desktop-cpu-core-i7-7820x-fp32.pkl", "rb") as f:
        df = pickle.load(f)
    api = API("./data_process/nas_201_api/NAS-Bench-201-v1_0-e61699.pth", verbose=False)
    import numpy as np

    data = {
        "key": [],
        "adj_matrix": [],
        "features": [],
        "flops": [],
        "params": [],
        "accuracy": [],
        "latency": [],
    }

    for i in trange(len(df)):
        try:
            key = np.asarray(df[i, :-1], dtype=np.int32)
            arch_str = get_arch_str_from_arch_vector(key)
            index = api.query_index_by_arch(arch_str)

            cost_info = api.get_cost_info(index, dataset="cifar10-valid")
            flops = float(cost_info["flops"])
            params = float(cost_info["params"])

            adj_matrix, label = get_matrix_and_ops(key)
            adj_matrix, features = get_adjacency_and_features(adj_matrix, label)
            features = np.argmax(features, axis=1)

            acc_info = api.get_more_info(
                index, dataset="cifar10-valid", iepoch=None, hp="200", is_random=False
            )

            accuracy = float(acc_info["test-accuracy"])
            latency = float(df[i, -1])

            # -------- 改成往 list 里 append --------
            data["key"].append(key)
            data["adj_matrix"].append(adj_matrix)
            data["features"].append(features)
            data["flops"].append(flops)
            data["params"].append(params)
            data["accuracy"].append(accuracy)
            data["latency"].append(latency)

        except Exception as e:
            print(f"[Warning] Skipped item {i} due to: {e}")

    for key in data:
        data[key] = np.array(data[key])
        print(f"{key} shape: {data[key].shape}")

    return data
